In [1]:
from IPython.core.display import display, HTML

from config import ERC721_ANALYSIS_DATASET_NAME, AZIMUTH_POINTS_TABLE_NAME, SNAPSHOT_TS
from src.utils_bigquery import get_df, create_table_from_df
from src.extractor_urbit import extract_point_data

SELECT_TRANSFERRED_TOKENS = True
DROP_TABLES = True
CREATE_TABLES = True

AZIMUTH_CONTRACT_ADDRESS = '0x223c067F8CF28ae173EE5CafEa60cA44C335fecB'.lower()
LINEAR_STAR_RELEASE_CONTRACT_ADDRESS= '0x86cd9cd0992F04231751E3761De45cEceA5d1801'.lower()
SPAWNED_EVENT_TOPIC = '0xb2d3a6e7a339f5c8ff96265e2f03a010a8541070f3744a247090964415081546'
ZERO_ADDRESS = '0x0000000000000000000000000000000000000000'
AZIMUTH_CONTRACT_CREATE_TS = '2018-11-27 00:00:00'

## Get List of Transferred Tokens

In [2]:
if SELECT_TRANSFERRED_TOKENS:
    query_1 = f'''
    SELECT cast(topics[ORDINAL(3)] AS INT64) as point
    FROM `bigquery-public-data.crypto_ethereum.logs`
    WHERE block_timestamp > '{AZIMUTH_CONTRACT_CREATE_TS}'
      AND block_timestamp < '{SNAPSHOT_TS}'
      AND address = '{AZIMUTH_CONTRACT_ADDRESS}'
      AND topics[ORDINAL(1)] = '{SPAWNED_EVENT_TOPIC}'
    '''

    azimuth_tokens_list = list(get_df(query=query_1).point)
    print(f'Number of Azimuth Tokens: {len(azimuth_tokens_list):>,}')

Number of Azimuth Tokens: 75,704


## Get Point Owners and Types

In [3]:
points_df = extract_point_data(point_list=azimuth_tokens_list, pool_size=10, chunk_size=64)

Processing 75704 points split into 1183 chunks on 10 processes:
Points 16456:29295616 extracted
Points 36822:12335 extracted
Points 5112320:25416 extracted
Points 12580:36164 extracted
Points 663:29395 extracted
Points 39167:7964 extracted
Points 43600:40261 extracted
Points 5516:51228 extracted
Points 21536:3134 extracted
Points 3634583221:2173961007 extracted
Points 29651:56311 extractedPoints 6422784:3670528 extracted

Points 36420:3870 extracted
Points 8220:54271 extracted
Points 3390:23075 extracted
Points 40517:36678 extracted
Points 12591:3030 extracted
Points 25672:9371904 extracted
Points 51484:21132 extracted
Points 4211541427:50235 extracted
Points 56567:9775 extracted
Points 3670784:16736 extracted
Points 23331:14625 extracted
Points 36934:56326 extracted
Points 50491:31009 extracted
Points 54783:64255 extracted
Points 4126:1617 extracted
Points 9372672:3350724536 extracted
Points 3283:105907456 extractedPoints 21388:27532 extracted

Points 10031:55229 extracted
Points 6451

In [4]:
display(HTML(points_df.head().to_html(index=False, notebook=True, show_dimensions=False)))

point,point_size,point_type,owner,parent_point
3634583221,2,planet,0xDd6621436046716b49A9b4c0Aafa929813171346,55459
71604,2,planet,0xf4661DAE4234c7C3f8d9559E3f4EF69a7e684115,1
650838831,2,planet,0xc548487148eF83ef6b0Dc3D2B7953A59c1Be89bb,9931
647116192,2,planet,0xc548487148eF83ef6b0Dc3D2B7953A59c1Be89bb,9874
3870818722,2,planet,0x2C12679C70e8059044DB933dDa1C8bbc4d49548F,59064


## Insert Point Data

In [5]:
create_table_from_df(source_df=points_df,
                     dataset_name=ERC721_ANALYSIS_DATASET_NAME,
                     table_name=AZIMUTH_POINTS_TABLE_NAME,
                     drop_existing_table=DROP_TABLES)

Table erc721_analysis:azimuth_points has been deleted.
Table erc721_analysis:azimuth_points has been created.


True